In [ ]:
# API utils libraries.
import requests
import json

# Data handling libraries
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np

# Data visualization libraries.
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

print("Setup Complete")


In [ ]:
# Array containing the list of existing media objects.
media_list = []

# Get Media API endpoint parameters.

# The total number of media objects we want to retrieve.
# For the purpose of our example, we'll retrieve 300 pictures.
total_media = 300
# Your Instagram user id.
ig_user_id = 'yyyy'
# API access token.
token = 'XXXX'
media_endpoint = 'https://graph.facebook.com/v7.0/%s/media?fields=timestamp,like_count,comments_count,media_type&access_token=%s&limit=%s' % (ig_user_id, token, total_media)

# Get the number of required media objects.
response = requests.get(media_endpoint)
response_json = response.json()

if "error" in response_json:
    raise Exception(response_json["error"]["message"]) 
        
media_list = response_json["data"]

In [ ]:
instagram_data_normalised = pd.json_normalize(media_list)
instagram_data_normalised.tail()

In [ ]:
instagram_data = instagram_data_normalised.copy()

# Remove videos from our dataset.
# instagram_data = instagram_data.drop(instagram_data[instagram_data.media_type != 'IMAGE'].index)

# Convert the timestamp to datetime type.
instagram_data['timestamp'] = pd.to_datetime(instagram_data['timestamp'], errors='coerce')

# Create an additional column to extract the day 
# of the week when the media object was uploaded.
instagram_data["weekday"] = instagram_data["timestamp"].dt.weekday

# Create an additional column to extract the time
# (hour) when the media object was uploaded.
instagram_data["hour"] = instagram_data["timestamp"].dt.hour

# Convert week days from numerical (0-6) to categorical values (Monday-Sunday).
weekday_categoricals = {
    "weekday": {
        0: "Monday",
        1: "Tuesday",
        2: "Wednesday",
        3: "Thursday",
        4: "Friday",
        5: "Saturday",
        6: "Sunday"
    }
}
instagram_data.replace(weekday_categoricals, inplace=True)

# Remove no longer needed columns.
del instagram_data['id']
del instagram_data['timestamp']
del instagram_data['comments_count']
del instagram_data['media_type']

instagram_data.head()

In [ ]:
# Pivot the dataframe to use hour as index (Y axis) and
# week days as labels (X axis). We could use just one
# pivot table (using aggfunc='mean') bu twe want to process
# the count table first.
instagram_data_pivot_count = instagram_data.pivot_table(
    index='hour',
    columns='weekday',
    values='like_count',
    aggfunc='count')


instagram_data_pivot_sum = instagram_data.pivot_table(
    index="hour",
    columns='weekday',
    values='like_count',
    aggfunc=np.sum)

# Remove from the dataset time slots with less than one
# uploaded picture.
instagram_data_pivot_count = instagram_data_pivot_count[instagram_data_pivot_count > 1]

# Calculate the picture likes mean per time slot.
instagram_data_pivot_mean = instagram_data_pivot_sum / instagram_data_pivot_count

instagram_data_pivot_mean.tail()


In [ ]:
# Initialise the plot configuration.
sns.set()
fig, ax = plt.subplots(figsize=(18,18))

# Set the graph to be displayed.
ax = sns.heatmap(cmap="Blues", data=instagram_data_pivot_mean, annot=True, xticklabels=["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"])

# Set other graph visualisation parameters.
pos, textvals = plt.yticks()
plt.xticks(fontsize="12")
plt.yticks(np.concatenate((pos , np.array([pos[len(pos) - 1] + 1]))) - 0.5, instagram_data_pivot_mean.index, rotation=0, fontsize="12", va="center")

ax.set_title('Instagram likes', fontsize=20, fontdict={})
ax.set_xlabel('Day of the week', fontsize=16)
ax.set_ylabel('Time', fontsize=16)

ax.set_facecolor((1, 1, 1))

# Display the Heatmap!
plt.show()